# 제네릭
- Kotlin은 컬렉션 사용 시 제네릭 타입 지정을 강제함 (Java는 선택사항)
-  Kotlin의 컴파일러는 컨텍스트를 통해 제네릭 타입을 추론할 수 있어, 명시적 타입 선언을 줄일 수 있습니다.

### 제네릭의 장점

- 컴파일 타임에 타입 검사 가능
- 코드가 더 명확해짐
- 캐스팅 횟수 감소



In [3]:
val list: MutableList<String> = mutableListOf("hello")
val liet = mutableListOf<String>()
// val list = mutableListOf() // 오류 발생!

// 타입 추론 활용
val list2 = mutableListOf("hello") // 타입이 자동으로 MutableList<String>로 추론됨

In [4]:
import java.math.BigDecimal

fun <T> printCollection(collection: Collection<T>) {
    for (item in collection) {
        println(item)
    }
}

val stringList = mutableListOf("hello", "world")
val numberList = mutableListOf(BigDecimal("33.45"), BigDecimal("350.99"))

printCollection(stringList)
printCollection(numberList)


hello
world
33.45
350.99


### 확장 함수

In [5]:
fun <T> List<T>.printCollection() {
    for (item in this) {
        println(item)
    }
}

stringList.printCollection()
numberList.printCollection()

hello
world
33.45
350.99


### 타입 매개변수 선언
- <T>: 함수명 앞에 위치하는 타입 매개변수 선언
- T는 관례적으로 사용되는 이름으로 "any Type"을 의미
- 여러 타입 매개변수 사용 가능: <T, U, V>

## 2. 타입 매개변수 제약(Type Parameter Constraints)
### 상한 제약(Upper Bound)
제네릭 타입으로 사용할 수 있는 타입을 제한할 수 있습니다.

In [6]:
// T 타입이 뭔지 알 수 없어 오류 발생
fun <T> convertToIntX(collection: Collection<T>) {
    for (item in collection) {
        //println(item) 오류!
    }
}

fun <T : Number> convertToInt(collection: Collection<T>) {
    for (item in collection) {
        println(item.toInt())
    }
}

val ints = listOf(1, 2, 3)
val shorts = listOf(1,3,4)
val floats = listOf(1234.3f, -3432.43f)

convertToInt(ints)
convertToInt(shorts)
convertToInt(floats)

val strings = listOf("1","2")
// convertToInt(strings) 오류남

1
2
3
1
3
4
1234
-3432


### 다중 상한 제약(where)
하나 이상의 상한을 지정할 때 where 절을 사용합니다.
- 여러 인터페이스는 가능하지만, 클래스는 하나만 지정 가능
  - `where T : Short, T : Int`는 불가능

In [16]:
fun <T> append(item1: T, item2: T)
        where T : CharSequence, T : Appendable {
    println("Result is: ${item1.append(item2)}")
}

// StringBuilder는 CharSequence와 Appendable 모두 구현
val sb1 = StringBuilder("Hello ")
val sb2 = StringBuilder("World")
append(sb1, sb2)

Result is: Hello World


## 3. null 가능성과 제네릭
제네릭은 기본적으로 null 가능성을 포함합니다.

In [8]:
// T의 상한은 암묵적으로 Any?입니다
fun <T> printCollection(collection: Collection<T>) {
    for (item in collection) {
        println(item) // T는 널 가능 타입도 허용
    }
}

val nullableShorts = listOf<Short?>(1, null, 3)
val nonNullStrings = listOf("a", "b", "c")

printCollection(nullableShorts) // 정상 작동
printCollection(nonNullStrings) // 정상 작동

1
null
3
a
b
c


### null 불가능 타입으로 제한
- 널 불가능 타입만 허용하려면 Any로 상한을 제한

In [9]:
fun <T : Any> printCollection(collection: Collection<T>) {
    for (item in collection) {
        println(item) // T는 널 불가능 타입만 허용
    }
}

val nullableShorts = listOf<Short?>(1, null, 3)
val nonNullStrings = listOf("a", "b", "c")

// printCollection(nullableShorts) // 컴파일 오류!
printCollection(nonNullStrings)    // 정상 작동

a
b
c


## 3. 타입 소거(Type Erasure)

In [10]:
val strings = listOf("str1", "str2")

// 컴파일 타임에 타입이 확실한 경우 - 가능
if (strings is List<String>) {
    println("This list contains strings")
}

// 런타임에 타입이 불확실한 경우 - 불가능
val listAny: Any = listOf("str1", "str2")
// if (listAny is List<String>) {} // 오류!

This list contains strings


### star projection
- 타입 인자를 확인할 수 없을때 `*`를 사용

In [11]:
val listAny: Any = listOf("str1", "str2")

// Star projection 사용으로 해결
if (listAny is List<*>) {
    println("Yes, this is a list")
}

Yes, this is a list


### 캐스팅과 위험성

In [15]:
// 안전하지 않은 캐스팅
val listAny1: Any = listOf("str1", "str2")

if (listAny1 is List<*>) {
    // Unchecked cast 경고 발생
    val strList = listAny1 as List<String>
    println(strList[1].replace("str", "string"))
}

//런타임 오류 위험
var listAny2: Any = listOf("str1", "str2")

// 나중에 다른 타입으로 변경
listAny2 = listOf(1, 2, 3) // 정수 리스트

if (listAny2 is List<*>) {
    val strList = listAny2 as List<String> // 컴파일은 됨
    println(strList[0].length) // 런타임 ClassCastException!
}

string2


java.lang.ClassCastException: class java.lang.Integer cannot be cast to class java.lang.String (java.lang.Integer and java.lang.String are in module java.base of loader 'bootstrap')